In [1]:
import env
import numpy as np
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from dqn_agent import Agent
from collections import deque

In [2]:
STATE_SIZE = (84, 84, 1)

ACTION_MAP = {
    0: "IDLE",
    1: "TURN_LEFT",
    2: "TURN_RIGHT",
    3: "LEFT",
    4: "RIGHT",
    5: "FORWARD",
    6: "BACKWARD",
    7: "SHOOT",
}
ACTION_SIZE = 8

In [3]:
environment = env.Environment('bevystein.exe', STATE_SIZE)

In [4]:
# while True:
#     environment.reset()
#     is_terminated = False
#     while not is_terminated:
#         (state, image) = environment.step("TURN_RIGHT")
#         is_terminated = state['is_terminated']
#         # print(state)


In [5]:
from model import VisualQNetwork

agent = Agent(state_size=STATE_SIZE, action_size=ACTION_SIZE, seed=0, double=True, priority_replay=False, q_network=VisualQNetwork)

#

In [6]:
def augment_state(frames, actions):
    action_t_minus_1, action_t = actions[-1], actions[0]
    pix_t_minus_1 = frames[0]
    pix_t = frames[1]
    pix_t_plus_1 = frames[2]
    
    action_t_minus_1 = np.ones((84, 84)) * action_t_minus_1
    action_t = np.ones((84, 84)) * action_t
           
    state = np.stack([
        pix_t_minus_1, 
        pix_t,
        pix_t_plus_1,
    ])
    
    return state

def dqn(n_episodes=2000, max_t=100, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    
    for i_episode in range(1, n_episodes+1):
        framebuffer = deque(maxlen=3)
        action_buffer = deque(maxlen=2)
        environment.reset()
        # get initial 3-frame observation
        state = np.squeeze(environment.visual_observations())
        for _ in range(0, 3):
            framebuffer.append(state)
        for _ in range(0, 2):
            action_buffer.append(0)
        
        state = augment_state(list(framebuffer), list(action_buffer))
        
        score = 0
        for t in range(max_t):
            try:
                action = agent.act(state, eps).astype(int)
                action_buffer.append(action)
                (result, screen) = environment.step(ACTION_MAP[action])
                framebuffer.append(np.squeeze(screen))
                next_state = augment_state(list(framebuffer), list(action_buffer))
                reward = result['reward']
                done = result['is_terminated']
                agent.step(state, action, reward, next_state, done)
                state = next_state
                score += reward
                if done: break 
            except:
                break
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
#             torch.save(agent.qnetwork_local.state_dict(), "visual_q_network.pth")
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))

    return scores

In [7]:
def train_dqn(n_episodes):
    scores = dqn(n_episodes=n_episodes)
    # plot the scores
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

In [8]:
train_dqn(n_episodes=2000)

Episode 3	Average Score: 3.33

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=7878): Max retries exceeded with url: /step (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019FC669E2E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))